In [42]:
import torch
import torch.nn as nn

In [81]:
def double_conv(input_channel, output_channel):
    conv = nn.Sequential(
           nn.Conv2d(input_channel, output_channel, kernel_size=3), 
           nn.ReLU(inplace=True),
           nn.Conv2d(output_channel, output_channel, kernel_size=3), 
           nn.ReLU(inplace=True),
           )              
    return conv                 
                         
def crop_image(tensor, target_tensor):
    target_size = target_tensor.size()[2]
    input_tensor = tensor.size()[2]                        
    delta = input_tensor - target_size
    delta = delta // 2
    return tensor[:,:,delta:input_tensor-delta, delta:input_tensor-delta]



class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        
        self.max_pool_3x3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.down_conv_1 = double_conv(1, 64)
        self.down_conv_2 = double_conv(64, 128)
        self.down_conv_3 = double_conv(128, 256)
        self.down_conv_4 = double_conv(256, 512)
        self.down_conv_5 = double_conv(512, 1024) 

        self.up_trans_1 = nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=2, stride=2)
        self.up_trans_2 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=2, stride=2)
        self.up_trans_3 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=2, stride=2)
        self.up_trans_4 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=2, stride=2)

        self.up_conv_1 = double_conv(1024, 512)
        self.up_conv_2 = double_conv(512, 256)
        self.up_conv_3 = double_conv(256, 128)
        self.up_conv_4 = double_conv(128, 64)

        self.out_conv = nn.Conv2d(in_channels=64, out_channels=2, kernel_size=1)

    def forward(self, image):
        # batch_size, channel, height, width
        # encoder part
        x1 = self.down_conv_1(image) #
        x2 = self.max_pool_3x3(x1)
        
        x3 = self.down_conv_2(x2) #
        x4 = self.max_pool_3x3(x3)

        x5 = self.down_conv_3(x4) #
        x6 = self.max_pool_3x3(x5)

        x7 = self.down_conv_4(x6) #
        x8 = self.max_pool_3x3(x7)

        x9 = self.down_conv_5(x8)

        # decoder part
        x = self.up_trans_1(x9)
        y = crop_image(x7, x)
        x = self.up_conv_1(torch.cat([x,y], 1)) # x and y will be concatenated on dimension 1 that represanted to Y axis.
        x = self.up_trans_2(x)
        y = crop_image(x5, x)
        x = self.up_conv_2(torch.cat([x,y], 1))

        x = self.up_trans_3(x)
        y = crop_image(x3, x)
        x = self.up_conv_3(torch.cat([x,y], 1))

        x = self.up_trans_4(x)
        y = crop_image(x1, x)
        x = self.up_conv_4(torch.cat([x,y], 1))

        print(x.size())
        out =  self.out_conv(x)

        return out 

if __name__ == "__main__":
    # test = torch.rand((1,2,572,572)) # 1 tensor var, içinde iki tane channel var bu channellar 572x572 lik iki adet resim gibi düşünülebilir.
    image = torch.rand((1,1,572,572))
    model = UNet()
    output = model(image)

torch.Size([1, 64, 388, 388])
